In [1]:
from bs4 import BeautifulSoup, Comment
import numpy as np
import pandas as pd
import requests
import time
import random
import re
from vardata import years, team_abbr, columns, teams, teams_wkly, weekly_columns

year = '2023'
week = '5'

In [2]:
url = 'https://www.pro-football-reference.com'

# function to convert output a dataframe as a csv
def df_to_csv(df, year, week):
    exec("df.to_csv('games_" + year + "_week_" + week + ".csv')")

In [3]:
def scrape_games(year, week):
    
    # use beautiful soup to return the html of the webpage
    r = requests.get(url + '/years/'+ year + '/games.htm')
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # grab the game results table from the webpage
    game_stats = soup.find_all('table')[0]

    # collect table rows
    stat_rows = game_stats.findAll('tr')
    
    # initalize 
    results = []
    n = 0
    
    # looping through the table rows to collect data
    for i in range(len(stat_rows)):
        if n >= 19:
            wait = random.randint(65, 120)
            time.sleep(wait)
            n = 0
        
        if stat_rows[i].find('th').getText() != week:
            continue
            
        row = []
        row.extend([col.getText() for col in stat_rows[i].findAll('td')])

#         if not row:
#             continue
#         elif row[1] == 'Playoffs':
#             break
        
        # navigate to the preview page and find the vegas line
        #-------------------------------------------------------------------------
        # get box score url
        link = stat_rows[i].find(href=re.compile("boxscores"))
        href=link.get('href')
        print(row)
        
        # use beautiful soup to return the html of the webpage
        r1 = requests.get(url + href)
        soup1 = BeautifulSoup(r1.content, 'html.parser')
        
        
        mydivs = soup1.find("div", class_="scorebox_meta")
        
        vegas_line = mydivs.find_next("div")

        while (vegas_line.text.split())[0] != "Vegas:":
            vegas_line = vegas_line.find_next("div")
        vegas_line = vegas_line.text
        
        favorite = ' '.join(vegas_line.split()[1:-1])
        favorite = teams_wkly[favorite]
        vegas_line = vegas_line.split()[-1]
        
        if vegas_line == 'Pick':
            vegas_line = 0
        else:
            vegas_line = float(vegas_line)
        #-------------------------------------------------------------------------
        
        # handling home / away team info being in different locations for each row
        if row[4] != '@':
            home_team = teams[row[3]]
            away_team = teams[row[5]]

            if row[3] == favorite:
                ht_vegas_line = vegas_line
            else:
                ht_vegas_line = abs(vegas_line)
        else:
            home_team = teams[row[5]]
            away_team = teams[row[3]]

            if row[3] == favorite:
                ht_vegas_line = abs(vegas_line)
            else:
                ht_vegas_line = vegas_line

        # appending necessary info to results array
        results.append([home_team, away_team, ht_vegas_line])

        n += 1
            
    return results

In [4]:
results = scrape_games(year, week)
exec('games_' + year + '_week_' + week + '''= pd.DataFrame(results, columns = ['home_team', 'away_team', 
            'ht_vegas_line'])''')
exec('df_to_csv(games_'+ year + '_week_' + week +', year, week)')

['Thu', '2023-10-05', '8:15PM', 'Chicago Bears', '@', 'Washington Commanders', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '9:30AM', 'Jacksonville Jaguars', '@', 'Buffalo Bills', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '1:00PM', 'Carolina Panthers', '@', 'Detroit Lions', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '1:00PM', 'Houston Texans', '@', 'Atlanta Falcons', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '1:00PM', 'New Orleans Saints', '@', 'New England Patriots', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '1:00PM', 'New York Giants', '@', 'Miami Dolphins', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '1:00PM', 'Tennessee Titans', '@', 'Indianapolis Colts', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '1:00PM', 'Baltimore Ravens', '@', 'Pittsburgh Steelers', 'preview', '', '', '', '', '', '']
['Sun', '2023-10-08', '4:05PM', 'Cincinnati Bengals', '@', 'Arizona Cardinals', 'preview', '', '',

In [7]:
def scrape_stats(team, year):
    
    print(team)
    # use beautiful soup to return the html of the webpage
    r = requests.get(url + '/teams/' + team + '/' + year + '.htm')
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # grab the team stats table from the team webpage
    team_stats = soup.find_all('table')[0]
    # grab the team conversions table from the team webpage
    team_conv = soup.find_all('table')[2]

    # Collect table rows from stats table
    stat_rows = team_stats.findAll('tr')[2:4]
    # Get stats from each row
    stats = []
    for i in range(len(stat_rows)):
        stats.extend([float(col.getText() or 0) for col in stat_rows[i].findAll('td')[:25]])

    # Collect table rows from conversions table
    conv_rows = team_conv.findAll('tr')[2:4]
    # Get stats from each row
    convs = []
    for i in range(len(conv_rows)):
        convs.extend([float(col.getText().strip('%') or 0) for col in conv_rows[i].findAll('td')])
   
    # combine all stats
    stats.extend(convs)
    # insert the team abbrev at beggining of array
    stats.insert(0, team)
    # print(stats)
    return stats

In [8]:
weekly_data = pd.DataFrame(columns = weekly_columns)

games = pd.read_csv('games_'+ year + '_week_' + week + '.csv')

n = 1

# loop through every game
for index in games.index:
    
    if n >= 26:
        time.sleep(65)
        n = 1

    # get home and away team indicies
    home_team = games['home_team'][index]
    away_team = games['away_team'][index]
    vegas_line = games['ht_vegas_line'][index]
    
    home_stats = scrape_stats(home_team, year)
    away_stats = scrape_stats(away_team, year)
    
    temp = [home_team, away_team]
    
    # append average of home stats with respect to away team stats
    for i in range(25):
        temp.append(np.mean((home_stats[i+1],away_stats[26+i])))
    for i in range(25):
        temp.append(np.mean((away_stats[i+1], home_stats[26+i])))
    for i in range(9):
        temp.append(np.mean((home_stats[51+i],away_stats[60+i])))
    for i in range(9):
        temp.append(np.mean((away_stats[51+i],home_stats[60+i])))
    
    weekly_data.loc[len(weekly_data)] = temp
    
    n+=2
    
weekly_data.to_csv('stats_' + year + '_week_' + week + '.csv')

was
chi
buf
jax
det
car
atl
htx
nwe
nor
mia
nyg
clt
oti
pit
rav
crd
cin
ram
phi
min
kan
den
nyj
sfo
dal
rai
gnb
